<h1 p style='text-align: center;'>Base Dados ANTAQ <h1>
<h2 p style='text-align: center;'> Gerção das Tabelas fatos: atracacao_fato e carga_fato <h2>
<h3 p style='text-align: center;'>Extração e transformação dos dados em Dataframes<h3>
<h3 p style='text-align: center;'>Questão 02<h3>
<h4 p style='text-align: center;'> Autor: Elifranio Alves Cruz <h4>
<h2 p style='text-align: left;'> Extração <h2>

In [1]:
#definições das bibliotecas
import pyspark
from pyspark import SparkContext

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile

from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Collinear Points").setMaster("local[4]") # inicializar o Spark unsando o contexto de 4 núcleo de trabalhos (jobs) 
#sc = SparkContext(conf=conf)    
from pyspark.rdd import RDD

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
dir(pyspark)

['Accumulator',
 'AccumulatorParam',
 'BarrierTaskContext',
 'BarrierTaskInfo',
 'BasicProfiler',
 'Broadcast',
 'HiveContext',
 'InheritableThread',
 'MarshalSerializer',
 'PickleSerializer',
 'Profiler',
 'RDD',
 'RDDBarrier',
 'Row',
 'SQLContext',
 'SparkConf',
 'SparkContext',
 'SparkFiles',
 'SparkJobInfo',
 'SparkStageInfo',
 'StatusTracker',
 'StorageLevel',
 'TaskContext',
 '_NoValue',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_globals',
 'accumulators',
 'broadcast',
 'cloudpickle',
 'conf',
 'context',
 'copy_func',
 'files',
 'find_spark_home',
 'java_gateway',
 'join',
 'keyword_only',
 'profiler',
 'rdd',
 'rddsampler',
 'resource',
 'resultiterable',
 'serializers',
 'shuffle',
 'since',
 'sql',
 'statcounter',
 'status',
 'storagelevel',
 'taskcontext',
 'traceback_utils',
 'types',
 'util',
 'version',
 'wraps']

In [5]:
from pyspark.sql import SparkSession

In [6]:
# criar uma sessão usanda para DF ( antaq) exemplo
spark = SparkSession.builder.appName("ANTAQ").getOrCreate()

In [7]:
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de acordos bilaterais.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
ano = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano+"/"+ano+"AcordosBilaterais.txt"):
        df = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano+"/"+ano+"AcordosBilaterais.txt", header=True, sep=';')
            


In [8]:
type(df)

pyspark.sql.dataframe.DataFrame

In [10]:
df.show(5)

+-----------------------+--------------------+----------------------+-------------------+---------+-----------------------+
|NacionalidadeEmbarcacao|Ano Acordo Bilateral|Total Acordo Bilateral|AcordoTipoNavegacao|     País|FlagEmbarqueDesembarque|
+-----------------------+--------------------+----------------------+-------------------+---------+-----------------------+
|          Não Informado|                2010|       1906211,1270003|        Longo Curso|ARGENTINA|                      1|
|                   null|                2010|       3160823,6960002|        Longo Curso|ARGENTINA|                      1|
|           Estrangeiras|                2010|       2158177,1970002|        Longo Curso|ARGENTINA|                      1|
|           Brasilieiras|                2010|       395849,45900001|        Longo Curso|ARGENTINA|                      1|
|                   null|                2010|               3011,25|           Interior|ARGENTINA|                      1|
+-------

In [48]:
#Pegar a primeira linha
df.first()

Row(NacionalidadeEmbarcacao='Não Informado', Ano Acordo Bilateral='2010', Total Acordo Bilateral='1906211,1270003', AcordoTipoNavegacao='Longo Curso', País='ARGENTINA', FlagEmbarqueDesembarque='1')

In [11]:
#pegar 5 cabeçalhos
df.head(5)

[Row(NacionalidadeEmbarcacao='Não Informado', Ano Acordo Bilateral='2010', Total Acordo Bilateral='1906211,1270003', AcordoTipoNavegacao='Longo Curso', País='ARGENTINA', FlagEmbarqueDesembarque='1'),
 Row(NacionalidadeEmbarcacao=None, Ano Acordo Bilateral='2010', Total Acordo Bilateral='3160823,6960002', AcordoTipoNavegacao='Longo Curso', País='ARGENTINA', FlagEmbarqueDesembarque='1'),
 Row(NacionalidadeEmbarcacao='Estrangeiras', Ano Acordo Bilateral='2010', Total Acordo Bilateral='2158177,1970002', AcordoTipoNavegacao='Longo Curso', País='ARGENTINA', FlagEmbarqueDesembarque='1'),
 Row(NacionalidadeEmbarcacao='Brasilieiras', Ano Acordo Bilateral='2010', Total Acordo Bilateral='395849,45900001', AcordoTipoNavegacao='Longo Curso', País='ARGENTINA', FlagEmbarqueDesembarque='1'),
 Row(NacionalidadeEmbarcacao=None, Ano Acordo Bilateral='2010', Total Acordo Bilateral='3011,25', AcordoTipoNavegacao='Interior', País='ARGENTINA', FlagEmbarqueDesembarque='1')]

In [12]:
# checar o nome das colunas
df.columns

['NacionalidadeEmbarcacao',
 'Ano Acordo Bilateral',
 'Total Acordo Bilateral',
 'AcordoTipoNavegacao',
 'País',
 'FlagEmbarqueDesembarque']

In [13]:
#checar os tipos de dados das colunas
df.dtypes

[('NacionalidadeEmbarcacao', 'string'),
 ('Ano Acordo Bilateral', 'string'),
 ('Total Acordo Bilateral', 'string'),
 ('AcordoTipoNavegacao', 'string'),
 ('País', 'string'),
 ('FlagEmbarqueDesembarque', 'string')]

In [14]:
# pegarv o esquema do dataframe
df.printSchema()

root
 |-- NacionalidadeEmbarcacao: string (nullable = true)
 |-- Ano Acordo Bilateral: string (nullable = true)
 |-- Total Acordo Bilateral: string (nullable = true)
 |-- AcordoTipoNavegacao: string (nullable = true)
 |-- País: string (nullable = true)
 |-- FlagEmbarqueDesembarque: string (nullable = true)



In [15]:
# verificar o número de linhas
df.count()

800

In [16]:
# checar o número de colunas
len(df.columns)

6

In [17]:
# checar a forma (linhas, colunas)
print(df.count(), len(df.columns) )

800 6


In [18]:
#### Tabela Acordo Bilateral######
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de acordos bilaterais.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
#Extraindo de forma manual no modelo de I/O do pyspark os dados da tabela de Acordo Bilateral

# criar uma sessão usanda para extração da tabela Acordo Bilateral
spark = SparkSession.builder.appName("ANTAQ.tb_ab").getOrCreate()


ano2020 = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"AcordosBilaterais.txt"):
        tb_ab2020 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"AcordosBilaterais.txt", header=True, sep=';')
            
ano2019 = "2019"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"AcordosBilaterais.txt"):
        tb_ab2019 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"AcordosBilaterais.txt", header=True, sep=';')
        
ano2018 = "2018"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"AcordosBilaterais.txt"):
        tb_ab2018 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"AcordosBilaterais.txt", header=True, sep=';')   
        
tb_ab = tb_ab2020.union(tb_ab2019.union(tb_ab2018))   

In [19]:
tb_ab2020.show(5)

+-----------------------+--------------------+----------------------+-------------------+---------+-----------------------+
|NacionalidadeEmbarcacao|Ano Acordo Bilateral|Total Acordo Bilateral|AcordoTipoNavegacao|     País|FlagEmbarqueDesembarque|
+-----------------------+--------------------+----------------------+-------------------+---------+-----------------------+
|          Não Informado|                2010|       1906211,1270003|        Longo Curso|ARGENTINA|                      1|
|                   null|                2010|       3160823,6960002|        Longo Curso|ARGENTINA|                      1|
|           Estrangeiras|                2010|       2158177,1970002|        Longo Curso|ARGENTINA|                      1|
|           Brasilieiras|                2010|       395849,45900001|        Longo Curso|ARGENTINA|                      1|
|                   null|                2010|               3011,25|           Interior|ARGENTINA|                      1|
+-------

In [20]:
tb_ab.show(5)

+-----------------------+--------------------+----------------------+-------------------+---------+-----------------------+
|NacionalidadeEmbarcacao|Ano Acordo Bilateral|Total Acordo Bilateral|AcordoTipoNavegacao|     País|FlagEmbarqueDesembarque|
+-----------------------+--------------------+----------------------+-------------------+---------+-----------------------+
|          Não Informado|                2010|       1906211,1270003|        Longo Curso|ARGENTINA|                      1|
|                   null|                2010|       3160823,6960002|        Longo Curso|ARGENTINA|                      1|
|           Estrangeiras|                2010|       2158177,1970002|        Longo Curso|ARGENTINA|                      1|
|           Brasilieiras|                2010|       395849,45900001|        Longo Curso|ARGENTINA|                      1|
|                   null|                2010|               3011,25|           Interior|ARGENTINA|                      1|
+-------

In [21]:
tb_ab2020.count() 

800

In [22]:
tb_ab2019.count()

749

In [23]:
 tb_ab2018.count()

694

In [24]:
tb_ab.count()

2243

In [25]:
#### Tabela Atracação######
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de atracação.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
#Extraindo de forma manual no modelo de I/O do pyspark os dados da tabela de Atracação

# criar uma sessão usanda para extração da tabela Atracacao
spark = SparkSession.builder.appName("ANTAQ.tb_atr").getOrCreate()

ano2020 = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Atracacao.txt"):
        tb_atr2020 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Atracacao.txt", header=True, sep=';')
            
ano2019 = "2019"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Atracacao.txt"):
        tb_atr2019 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Atracacao.txt", header=True, sep=';')
        
ano2018 = "2018"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Atracacao.txt"):
        tb_atr2018 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Atracacao.txt", header=True, sep=';')   
        
tb_atr = tb_atr2020.union(tb_atr2019.union(tb_atr2018))   

In [26]:
tb_atr2018.first()

Row(IDAtracacao='2462', CDTUP='BR', IDBerco=None, Berço=None, Porto Atracação='n/a', Apelido Instalação Portuária='n/a', Complexo Portuário='Não Classificado', Tipo da Autoridade Portuária='Porto Público', Data Atracação=None, Data Chegada=None, Data Desatracação='01/09/2018 00:00:00', Data Início Operação=None, Data Término Operação=None, Ano='2018', Mes='set', Tipo de Operação=None, Tipo de Navegação da Atracação=None, Nacionalidade do Armador=None, FlagMCOperacaoAtracacao='1', Terminal=None, Município=None, UF=None, SGUF=None, Região Geográfica=None, Nº da Capitania=None, Nº do IMO=None)

In [27]:
tb_atr2018.head()

Row(IDAtracacao='2462', CDTUP='BR', IDBerco=None, Berço=None, Porto Atracação='n/a', Apelido Instalação Portuária='n/a', Complexo Portuário='Não Classificado', Tipo da Autoridade Portuária='Porto Público', Data Atracação=None, Data Chegada=None, Data Desatracação='01/09/2018 00:00:00', Data Início Operação=None, Data Término Operação=None, Ano='2018', Mes='set', Tipo de Operação=None, Tipo de Navegação da Atracação=None, Nacionalidade do Armador=None, FlagMCOperacaoAtracacao='1', Terminal=None, Município=None, UF=None, SGUF=None, Região Geográfica=None, Nº da Capitania=None, Nº do IMO=None)

In [28]:
tb_atr2018.show(5)

+-----------+-----+-------+-----+---------------+----------------------------+------------------+----------------------------+--------------+------------+-------------------+--------------------+---------------------+----+---+----------------+------------------------------+------------------------+-----------------------+--------+---------+----+----+-----------------+---------------+---------+
|IDAtracacao|CDTUP|IDBerco|Berço|Porto Atracação|Apelido Instalação Portuária|Complexo Portuário|Tipo da Autoridade Portuária|Data Atracação|Data Chegada|  Data Desatracação|Data Início Operação|Data Término Operação| Ano|Mes|Tipo de Operação|Tipo de Navegação da Atracação|Nacionalidade do Armador|FlagMCOperacaoAtracacao|Terminal|Município|  UF|SGUF|Região Geográfica|Nº da Capitania|Nº do IMO|
+-----------+-----+-------+-----+---------------+----------------------------+------------------+----------------------------+--------------+------------+-------------------+--------------------+-----------

In [29]:
tb_atr2020.count()

76284

In [30]:
tb_atr2019.count()

77208

In [31]:
tb_atr2018.count()

76082

In [32]:
tb_atr.count()

229574

In [33]:
#### Tabela Carga######
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de Carga.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
#Extraindo de forma manual no modelo de I/O do pyspark os dados da tabela de Carga

# criar uma sessão usanda para extração da tabela Carga
spark = SparkSession.builder.appName("ANTAQ.tb_carga").getOrCreate()

ano2020 = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Carga.txt"):
        tb_carga2020 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Carga.txt", header=True, sep=';')
            
ano2019 = "2019"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Carga.txt"):
        tb_carga2019 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Carga.txt", header=True, sep=';')
        
ano2018 = "2018"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Carga.txt"):
        tb_carga2018 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Carga.txt", header=True, sep=';')   
        
tb_carga = tb_carga2020.union(tb_carga2019.union(tb_carga2018))   

In [34]:
tb_carga2020.first()

Row(IDCarga='25376365', IDAtracacao='1048499', Origem='BR200', Destino='BRBEL', CDMercadoria='2710', Tipo Operação da Carga='Interior', Carga Geral Acondicionamento=None, ConteinerEstado=None, Tipo Navegação='Interior', FlagAutorizacao='S', FlagCabotagem='0', FlagCabotagemMovimentacao='0', FlagConteinerTamanho=None, FlagLongoCurso='0', FlagMCOperacaoCarga='1', FlagOffshore='0', FlagTransporteViaInterioir='1', Percurso Transporte em vias Interiores='Interior de percurso não identificado', Percurso Transporte Interiores='Navegação Interior', STNaturezaCarga='Exclusivo', STSH2='Exclusivo', STSH4='Exclusivo', Natureza da Carga='Granel Líquido e Gasoso', Sentido='Desembarcados', TEU='0', QTCarga='0', VLPesoCargaBruta='1005,32')

In [35]:
tb_carga2020.select('IDCarga','IDAtracacao', 'Origem','Destino','Tipo Navegação').show(5) #Fica fora do escopo devido a quantidade colunas ser maior que a capacidade de resolução

+--------+-----------+-------+-------+--------------+
| IDCarga|IDAtracacao| Origem|Destino|Tipo Navegação|
+--------+-----------+-------+-------+--------------+
|25376365|    1048499|  BR200|  BRBEL|      Interior|
|25377972|    1048717|  USMOB|BRRJ016|   Longo Curso|
|25378091|    1048758|BRRS008|BRRS003|      Interior|
|25381355|    1048861|  BRBEL|  BRVDC|      Interior|
|25381441|    1048922|BRPA001|  IEAUG|   Longo Curso|
+--------+-----------+-------+-------+--------------+
only showing top 5 rows



In [36]:
tb_carga2020.count()

2096101

In [37]:
tb_carga2019.count()

2018617

In [38]:
tb_carga2018.count()

2087525

In [39]:
tb_carga.count()

6202243

In [40]:
#### Tabela Carga_Conteinerizada ######
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de Carga_Conteinerizada.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
#Extraindo de forma manual no modelo de I/O do pyspark os dados da tabela de Carga_Conteinerizada

# criar uma sessão usanda para extração da tabela Carga_Conteinerizada
spark = SparkSession.builder.appName("ANTAQ.tb_carga_contz").getOrCreate()

ano2020 = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Carga_Conteinerizada.txt"):
        tb_carga_contz2020 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Carga_Conteinerizada.txt", header=True, sep=';')
            
ano2019 = "2019"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Carga_Conteinerizada.txt"):
        tb_carga_contz2019 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Carga_Conteinerizada.txt", header=True, sep=';')
        
ano2018 = "2018"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Carga_Conteinerizada.txt"):
        tb_carga_contz2018 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Carga_Conteinerizada.txt", header=True, sep=';')   
        
tb_carga_contz = tb_carga_contz2020.union(tb_carga_contz2019.union(tb_carga_contz2018)) 

In [41]:
tb_carga_contz2020.show(5)

+--------+--------------------------+-------------------------+
| IDCarga|CDMercadoriaConteinerizada|VLPesoCargaConteinerizada|
+--------+--------------------------+-------------------------+
|25506812|                      0101|                     17,8|
|25506832|                      0101|                    14,12|
|25507093|                      0101|                     17,8|
|25507107|                      0101|                    36,44|
|25507177|                      0101|                     0,92|
+--------+--------------------------+-------------------------+
only showing top 5 rows



In [43]:
tb_carga_contz.show(5)

+--------+--------------------------+-------------------------+
| IDCarga|CDMercadoriaConteinerizada|VLPesoCargaConteinerizada|
+--------+--------------------------+-------------------------+
|25506812|                      0101|                     17,8|
|25506832|                      0101|                    14,12|
|25507093|                      0101|                     17,8|
|25507107|                      0101|                    36,44|
|25507177|                      0101|                     0,92|
+--------+--------------------------+-------------------------+
only showing top 5 rows



In [44]:
tb_carga_contz2020.count()

3570103

In [45]:
tb_carga_contz2019.count()

3500499

In [46]:
tb_carga_contz2018.count()

3651101

In [47]:
tb_carga_contz.count()

10721703

In [48]:
#### Tabela Carga_Regiao ######
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de Carga_Regiao.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
#Extraindo de forma manual no modelo de I/O do pyspark os dados da tabela de Carga_Regiao

# criar uma sessão usanda para extração da tabela Carga_Regiao
spark = SparkSession.builder.appName("ANTAQ.tb_carga_reg").getOrCreate()

ano2020 = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Carga_Regiao.txt"):
        tb_carga_reg2020 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"Carga_Regiao.txt", header=True, sep=';')
            
ano2019 = "2019"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Carga_Regiao.txt"):
        tb_carga_reg2019 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"Carga_Regiao.txt", header=True, sep=';')
        
ano2018 = "2018"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Carga_Regiao.txt"):
        tb_carga_reg2018 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"Carga_Regiao.txt", header=True, sep=';')   
        
tb_carga_reg = tb_carga_reg2020.union(tb_carga_reg2019.union(tb_carga_reg2018)) 



In [49]:
tb_carga_reg2020.show(5)

+--------+--------------------+----------------+
| IDCarga| Região Hidrográfica|ValorMovimentado|
+--------+--------------------+----------------+
|25376365|                null|         1005,32|
|25381441|Região Hidrográfi...|           55000|
|25381442|Região Hidrográfi...|         1334,24|
|25381556|Região Hidrográfi...|         1403,95|
|25381562|Região Hidrográfi...|         1686,19|
+--------+--------------------+----------------+
only showing top 5 rows



In [50]:
tb_carga_reg.show(5)

+--------+--------------------+----------------+
| IDCarga| Região Hidrográfica|ValorMovimentado|
+--------+--------------------+----------------+
|25376365|                null|         1005,32|
|25381441|Região Hidrográfi...|           55000|
|25381442|Região Hidrográfi...|         1334,24|
|25381556|Região Hidrográfi...|         1403,95|
|25381562|Região Hidrográfi...|         1686,19|
+--------+--------------------+----------------+
only showing top 5 rows



In [51]:
tb_carga_reg2020.count()

506791

In [52]:
tb_carga_reg2019.count()

482857

In [53]:
tb_carga_reg2018.count()

563870

In [54]:
tb_carga_reg.count()

1553518

In [55]:
#### Tabela TemposAtracacao ######
#Pode-se alternar 2018, 2019 e 2020 de forma manual de acordo com os arquivos nos diretórios, neste caso para 
# a tabela de TemposAtracacao.Ex.  ano = 2018; ou ano = 2019 ou ano = 2020
#Extraindo de forma manual no modelo de I/O do pyspark os dados da tabela de TemposAtracacao

# criar uma sessão usanda para extração da tabela TemposAtracacao
spark = SparkSession.builder.appName("ANTAQ.tb_temp_atr").getOrCreate()

ano2020 = "2020"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"TemposAtracacao.txt"):
        tb_temp_atr2020 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2020+"/"+ano2020+"TemposAtracacao.txt", header=True, sep=';')
            
ano2019 = "2019"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"TemposAtracacao.txt"):
        tb_temp_atr2019 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2019+"/"+ano2019+"TemposAtracacao.txt", header=True, sep=';')
        
ano2018 = "2018"
   # verifica se o arquivo existe no dietório
if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"TemposAtracacao.txt"):
        tb_temp_atr2018 = spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+ano2018+"/"+ano2018+"TemposAtracacao.txt", header=True, sep=';')   
        
tb_temp_atr = tb_temp_atr2020.union(tb_temp_atr2019.union(tb_temp_atr2018))


In [56]:
tb_temp_atr2020.show(5)

+-----------+----------------+----------------+---------------+-------------------+---------------+---------------+
|IDAtracacao|TEsperaAtracacao| TEsperaInicioOp|      TOperacao|TEsperaDesatracacao|      TAtracado|       TEstadia|
+-----------+----------------+----------------+---------------+-------------------+---------------+---------------+
|    1117343|0,48333333333721|0,18333333340706|39,249999999942|   0,25000000011642|39,683333333465|40,166666666802|
|    1117350|0,48333333333721|0,18333333323244|147,08333333343|   0,24999999994179|              0|147,99999999994|
|    1117358|0,48333333333721|0,18333333340706|13,249999999884|   0,25000000011642|13,683333333407|14,166666666744|
|    1117341|0,48333333333721|0,18333333323244|          18,75|   0,25000000011642|19,183333333349|19,666666666686|
|    1117352|0,48333333333721|0,18333333340706|15,999999999942|   0,24999999994179|16,433333333291|16,916666666628|
+-----------+----------------+----------------+---------------+---------

In [57]:
tb_temp_atr.show(5)

+-----------+----------------+----------------+---------------+-------------------+---------------+---------------+
|IDAtracacao|TEsperaAtracacao| TEsperaInicioOp|      TOperacao|TEsperaDesatracacao|      TAtracado|       TEstadia|
+-----------+----------------+----------------+---------------+-------------------+---------------+---------------+
|    1117343|0,48333333333721|0,18333333340706|39,249999999942|   0,25000000011642|39,683333333465|40,166666666802|
|    1117350|0,48333333333721|0,18333333323244|147,08333333343|   0,24999999994179|              0|147,99999999994|
|    1117358|0,48333333333721|0,18333333340706|13,249999999884|   0,25000000011642|13,683333333407|14,166666666744|
|    1117341|0,48333333333721|0,18333333323244|          18,75|   0,25000000011642|19,183333333349|19,666666666686|
|    1117352|0,48333333333721|0,18333333340706|15,999999999942|   0,24999999994179|16,433333333291|16,916666666628|
+-----------+----------------+----------------+---------------+---------

In [58]:
tb_temp_atr2020.count()

74941

In [59]:
tb_temp_atr2019.count()

76536

In [60]:
tb_temp_atr2018.count()

75642

In [61]:
tb_temp_atr.count()

227119

<h2>Transformação <h2>

In [62]:
# Dicionário das tabelas Extraidas
#tb_ab  ; tb_atr ; tb_carga ; tb_carga_contz; tb_carga_reg; tb_temp_atr
###############
# tb_ab - Tabela Acordos Bilaterais
# tb_atr - Tabela Atracacao
# tb_carga - Tabela Carga
# tb_carga_contz - Tabela Carga_Conteinerizada
# tb_carga_reg - Tabela Carga_Regiao
# tb_temp_atr - Tabela TemposAtracacao
###############

# Dicionário dos atributos a serem extraidos e transformados

#At_atracacao_fato = ["IDAtracacao" , " Tipo de Navegação da Atracação" , " CDTUP" , " Nacionalidade do Armador" , 
#                     " IDBerco" , " FlagMCOperacaoAtracacao" , " Berço Terminal" , "Porto Atracação" , " Município" , 
#                     " Apelido Instalação Portuária" , " UF" , " Complexo Portuário" , " SGUF" , 
#                     " Tipo da Autoridade Portuária" , "Região Geográfica" , " Data Atracação" , " Nº da Capitania" , 
#                     " Data Chegada" , " Nº do IMO" , " Data Desatracação" , " TEsperaAtracacao" , "Data Início Operação",
#                     " TEsperaInicioOp" , " Data Término Operação" , " TOperacao" , " Ano da data de início da operação" ,
#                     " TEsperaDesatracacao" , "Mês da data de início da operação" , " TAtracado" , " Tipo de Operação" , 
#                     " TEstadia"]
##############################
# carga_fato = [IDCarga" , " FlagTransporteViaInterioir" , " IDAtracacao" , " Percurso Transporte em vias Interiores" , 
#                " Origem Percurso Transporte Interiores" , "Destino" , " STNaturezaCarga" , 
#                " CDMercadoria (Para carga conteinerizadainformar código das mercadorias dentrodo contêiner.)" , " STSH2" , 
#                " Tipo Operação da Carga" , " STSH4" , " Carga Geral Acondicionamento" , " Natureza da Carga" ,
#                " ConteinerEstado" , " Sentido" , " Tipo Navegação" , " TEU" , "FlagAutorizacao" , " QTCarga" , 
#                " FlagCabotagem" , " VLPesoCargaBruta" , " FlagCabotagemMovimentacao" , 
#                " Ano da data de início da operação da atracação" , " FlagConteinerTamanho" , 
#                " Mês da data de início da operação da atracação" , " FlagLongoCurso" , " Porto Atracação" , 
#                " FlagMCOperacaoCarga" , " SGUF" , " FlagOffshore" , 
#                "  Peso líquido da carga (Carga não conteinerizada = Peso bruto e Carga conteinerizada = Peso sem contêiner)]




In [63]:
tb_ab.head()

Row(NacionalidadeEmbarcacao='Não Informado', Ano Acordo Bilateral='2010', Total Acordo Bilateral='1906211,1270003', AcordoTipoNavegacao='Longo Curso', País='ARGENTINA', FlagEmbarqueDesembarque='1')

In [64]:
tb_ab.columns

['NacionalidadeEmbarcacao',
 'Ano Acordo Bilateral',
 'Total Acordo Bilateral',
 'AcordoTipoNavegacao',
 'País',
 'FlagEmbarqueDesembarque']

In [65]:
tb_atr.head()

Row(IDAtracacao='3060', CDTUP='BR', IDBerco=None, Berço=None, Porto Atracação='n/a', Apelido Instalação Portuária='n/a', Complexo Portuário='Não Classificado', Tipo da Autoridade Portuária='Porto Público', Data Atracação=None, Data Chegada=None, Data Desatracação='01/08/2020 00:00:00', Data Início Operação=None, Data Término Operação=None, Ano='2020', Mes='ago', Tipo de Operação=None, Tipo de Navegação da Atracação=None, Nacionalidade do Armador=None, FlagMCOperacaoAtracacao='1', Terminal=None, Município=None, UF=None, SGUF=None, Região Geográfica=None, Nº da Capitania=None, Nº do IMO=None)

In [66]:
tb_atr.columns

['IDAtracacao',
 'CDTUP',
 'IDBerco',
 'Berço',
 'Porto Atracação',
 'Apelido Instalação Portuária',
 'Complexo Portuário',
 'Tipo da Autoridade Portuária',
 'Data Atracação',
 'Data Chegada',
 'Data Desatracação',
 'Data Início Operação',
 'Data Término Operação',
 'Ano',
 'Mes',
 'Tipo de Operação',
 'Tipo de Navegação da Atracação',
 'Nacionalidade do Armador',
 'FlagMCOperacaoAtracacao',
 'Terminal',
 'Município',
 'UF',
 'SGUF',
 'Região Geográfica',
 'Nº da Capitania',
 'Nº do IMO']

In [67]:
tb_carga.head()

Row(IDCarga='25376365', IDAtracacao='1048499', Origem='BR200', Destino='BRBEL', CDMercadoria='2710', Tipo Operação da Carga='Interior', Carga Geral Acondicionamento=None, ConteinerEstado=None, Tipo Navegação='Interior', FlagAutorizacao='S', FlagCabotagem='0', FlagCabotagemMovimentacao='0', FlagConteinerTamanho=None, FlagLongoCurso='0', FlagMCOperacaoCarga='1', FlagOffshore='0', FlagTransporteViaInterioir='1', Percurso Transporte em vias Interiores='Interior de percurso não identificado', Percurso Transporte Interiores='Navegação Interior', STNaturezaCarga='Exclusivo', STSH2='Exclusivo', STSH4='Exclusivo', Natureza da Carga='Granel Líquido e Gasoso', Sentido='Desembarcados', TEU='0', QTCarga='0', VLPesoCargaBruta='1005,32')

In [68]:
tb_carga.columns

['IDCarga',
 'IDAtracacao',
 'Origem',
 'Destino',
 'CDMercadoria',
 'Tipo Operação da Carga',
 'Carga Geral Acondicionamento',
 'ConteinerEstado',
 'Tipo Navegação',
 'FlagAutorizacao',
 'FlagCabotagem',
 'FlagCabotagemMovimentacao',
 'FlagConteinerTamanho',
 'FlagLongoCurso',
 'FlagMCOperacaoCarga',
 'FlagOffshore',
 'FlagTransporteViaInterioir',
 'Percurso Transporte em vias Interiores',
 'Percurso Transporte Interiores',
 'STNaturezaCarga',
 'STSH2',
 'STSH4',
 'Natureza da Carga',
 'Sentido',
 'TEU',
 'QTCarga',
 'VLPesoCargaBruta']

In [69]:
tb_carga_contz.head()

Row(IDCarga='25506812', CDMercadoriaConteinerizada='0101', VLPesoCargaConteinerizada='17,8')

In [70]:
tb_carga_contz.columns

['IDCarga', 'CDMercadoriaConteinerizada', 'VLPesoCargaConteinerizada']

In [71]:
tb_carga_reg.head()

Row(IDCarga='25376365', Região Hidrográfica=None, ValorMovimentado='1005,32')

In [72]:
tb_carga_reg.columns

['IDCarga', 'Região Hidrográfica', 'ValorMovimentado']

In [73]:
tb_temp_atr.head()

Row(IDAtracacao='1117343', TEsperaAtracacao='0,48333333333721', TEsperaInicioOp='0,18333333340706', TOperacao='39,249999999942', TEsperaDesatracacao='0,25000000011642', TAtracado='39,683333333465', TEstadia='40,166666666802')

In [74]:
tb_temp_atr.columns

['IDAtracacao',
 'TEsperaAtracacao',
 'TEsperaInicioOp',
 'TOperacao',
 'TEsperaDesatracacao',
 'TAtracado',
 'TEstadia']

In [75]:
ano = ["2018","2019","2020"]
for i in ano:
    print(i)

2018
2019
2020


In [76]:
tb_carga_reg.head()

Row(IDCarga='25376365', Região Hidrográfica=None, ValorMovimentado='1005,32')

In [77]:
tb_carga.columns

['IDCarga',
 'IDAtracacao',
 'Origem',
 'Destino',
 'CDMercadoria',
 'Tipo Operação da Carga',
 'Carga Geral Acondicionamento',
 'ConteinerEstado',
 'Tipo Navegação',
 'FlagAutorizacao',
 'FlagCabotagem',
 'FlagCabotagemMovimentacao',
 'FlagConteinerTamanho',
 'FlagLongoCurso',
 'FlagMCOperacaoCarga',
 'FlagOffshore',
 'FlagTransporteViaInterioir',
 'Percurso Transporte em vias Interiores',
 'Percurso Transporte Interiores',
 'STNaturezaCarga',
 'STSH2',
 'STSH4',
 'Natureza da Carga',
 'Sentido',
 'TEU',
 'QTCarga',
 'VLPesoCargaBruta']

In [78]:
tb_atr.columns

['IDAtracacao',
 'CDTUP',
 'IDBerco',
 'Berço',
 'Porto Atracação',
 'Apelido Instalação Portuária',
 'Complexo Portuário',
 'Tipo da Autoridade Portuária',
 'Data Atracação',
 'Data Chegada',
 'Data Desatracação',
 'Data Início Operação',
 'Data Término Operação',
 'Ano',
 'Mes',
 'Tipo de Operação',
 'Tipo de Navegação da Atracação',
 'Nacionalidade do Armador',
 'FlagMCOperacaoAtracacao',
 'Terminal',
 'Município',
 'UF',
 'SGUF',
 'Região Geográfica',
 'Nº da Capitania',
 'Nº do IMO']

<h1 p style='text-align: center;'> Tabela atracacao_fato <h1>

In [79]:
######## Tabela atracacao_fato#######
#Criação da tabeça atracacao_fato
# Campo comun entre as duas tabelas (tb_atr.IDAtracacao == tb_temp_atr.IDAtracacao)
# Inner - Combina os valores comuns dos mesmos campos nas duas tabelas (Intersecção)

# criar uma sessão usanda para extração da tabela atracacao_fato
spark = SparkSession.builder.appName("ANTAQ.atracacao_fato").getOrCreate()

atracacao_fato = tb_atr.join(tb_temp_atr, tb_atr.IDAtracacao == tb_temp_atr.IDAtracacao, "inner" )
  



In [80]:
tb_atr.count()

229574

In [81]:
tb_temp_atr.count()

227119

In [82]:
atracacao_fato.count()

227119

In [83]:
atracacao_fato.columns

['IDAtracacao',
 'CDTUP',
 'IDBerco',
 'Berço',
 'Porto Atracação',
 'Apelido Instalação Portuária',
 'Complexo Portuário',
 'Tipo da Autoridade Portuária',
 'Data Atracação',
 'Data Chegada',
 'Data Desatracação',
 'Data Início Operação',
 'Data Término Operação',
 'Ano',
 'Mes',
 'Tipo de Operação',
 'Tipo de Navegação da Atracação',
 'Nacionalidade do Armador',
 'FlagMCOperacaoAtracacao',
 'Terminal',
 'Município',
 'UF',
 'SGUF',
 'Região Geográfica',
 'Nº da Capitania',
 'Nº do IMO',
 'IDAtracacao',
 'TEsperaAtracacao',
 'TEsperaInicioOp',
 'TOperacao',
 'TEsperaDesatracacao',
 'TAtracado',
 'TEstadia']

In [106]:
#Criação da Tabela 'atracacao_fato.csv' no diretório atual da execução do Jupyter / se desejar outro Path basta colocar na string
atracacao_fato.toPandas().to_csv('atracacao_fato.csv')

In [84]:
tb_carga.columns

['IDCarga',
 'IDAtracacao',
 'Origem',
 'Destino',
 'CDMercadoria',
 'Tipo Operação da Carga',
 'Carga Geral Acondicionamento',
 'ConteinerEstado',
 'Tipo Navegação',
 'FlagAutorizacao',
 'FlagCabotagem',
 'FlagCabotagemMovimentacao',
 'FlagConteinerTamanho',
 'FlagLongoCurso',
 'FlagMCOperacaoCarga',
 'FlagOffshore',
 'FlagTransporteViaInterioir',
 'Percurso Transporte em vias Interiores',
 'Percurso Transporte Interiores',
 'STNaturezaCarga',
 'STSH2',
 'STSH4',
 'Natureza da Carga',
 'Sentido',
 'TEU',
 'QTCarga',
 'VLPesoCargaBruta']

In [85]:
tb_atr.columns

['IDAtracacao',
 'CDTUP',
 'IDBerco',
 'Berço',
 'Porto Atracação',
 'Apelido Instalação Portuária',
 'Complexo Portuário',
 'Tipo da Autoridade Portuária',
 'Data Atracação',
 'Data Chegada',
 'Data Desatracação',
 'Data Início Operação',
 'Data Término Operação',
 'Ano',
 'Mes',
 'Tipo de Operação',
 'Tipo de Navegação da Atracação',
 'Nacionalidade do Armador',
 'FlagMCOperacaoAtracacao',
 'Terminal',
 'Município',
 'UF',
 'SGUF',
 'Região Geográfica',
 'Nº da Capitania',
 'Nº do IMO']

In [86]:
tb_carga_contz.columns

['IDCarga', 'CDMercadoriaConteinerizada', 'VLPesoCargaConteinerizada']

In [87]:
#teste filtro
tb_atr_filtro = tb_atr.select('IDAtracacao','Ano','Mes','Porto Atracação', 'SGUF')

In [88]:
tb_atr_filtro.show()

+-----------+----+---+---------------+----+
|IDAtracacao| Ano|Mes|Porto Atracação|SGUF|
+-----------+----+---+---------------+----+
|       3060|2020|ago|            n/a|null|
|       3061|2020|ago|            n/a|null|
|       3073|2020|ago|            n/a|null|
|       3067|2020|ago|            n/a|null|
|       3059|2020|jul|            n/a|null|
|       3114|2020|set|            n/a|null|
|       3088|2020|ago|            n/a|null|
|       3112|2020|set|            n/a|null|
|       3062|2020|ago|            n/a|null|
|       3115|2020|set|            n/a|null|
|       3113|2020|set|            n/a|null|
|       3066|2020|ago|            n/a|null|
|       3052|2020|jul|            n/a|null|
|       3049|2020|jul|            n/a|null|
|       3057|2020|jul|            n/a|null|
|       3056|2020|jul|            n/a|null|
|       3197|2020|dez|            n/a|null|
|       3050|2020|jul|            n/a|null|
|       3063|2020|ago|            n/a|null|
|       3065|2020|ago|          

In [89]:
#Teste Carga coneinerizada para pegar o valor do conteinerizado
tb_carga_contz_filtro = tb_carga_contz.select('IDCarga','VLPesoCargaConteinerizada')

In [90]:
tb_carga_contz_filtro.columns

['IDCarga', 'VLPesoCargaConteinerizada']

In [91]:
#Teste Carga para pegar o valor da carga bruta
tb_carga_filtro = tb_carga.select('IDCarga','VLPesoCargaBruta')

In [92]:
# Junção dos filtros com informações de cargas bruta e conteinrizada
# tb_carga_contz_filtro.IDCarga == tb_carga_filtro.IDCarga, "inner" == ['IDCarga']
tb_carga_liqui1 = tb_carga_filtro.join(tb_carga_contz_filtro, ['IDCarga'] )

In [93]:
tb_carga_liqui1.columns

['IDCarga', 'VLPesoCargaBruta', 'VLPesoCargaConteinerizada']

In [94]:
tb_carga_liqui2 = tb_carga_liqui1.withColumn('Peso Liquido',tb_carga_liqui1['VLPesoCargaBruta'] + tb_carga_liqui1['VLPesoCargaBruta'] )


In [95]:
tb_carga_liqui2.columns

['IDCarga', 'VLPesoCargaBruta', 'VLPesoCargaConteinerizada', 'Peso Liquido']

In [96]:
# Teste para pegar a informação do peso liquido
tb_carga_liqui = tb_carga_liqui2.select('IDCarga','Peso Liquido')

In [97]:
tb_carga_liqui.columns

['IDCarga', 'Peso Liquido']

In [98]:
#teste
#Procedimentos de junções finais
# tb_carga.IDAtracacao == tb_atr_filtro.IDAtracacao, "inner" == ['IDAtracacao']
carga_fato_tmp1 = tb_carga.join(tb_atr_filtro, ['IDAtracacao'] )


In [99]:
carga_fato_tmp1.columns

['IDAtracacao',
 'IDCarga',
 'Origem',
 'Destino',
 'CDMercadoria',
 'Tipo Operação da Carga',
 'Carga Geral Acondicionamento',
 'ConteinerEstado',
 'Tipo Navegação',
 'FlagAutorizacao',
 'FlagCabotagem',
 'FlagCabotagemMovimentacao',
 'FlagConteinerTamanho',
 'FlagLongoCurso',
 'FlagMCOperacaoCarga',
 'FlagOffshore',
 'FlagTransporteViaInterioir',
 'Percurso Transporte em vias Interiores',
 'Percurso Transporte Interiores',
 'STNaturezaCarga',
 'STSH2',
 'STSH4',
 'Natureza da Carga',
 'Sentido',
 'TEU',
 'QTCarga',
 'VLPesoCargaBruta',
 'Ano',
 'Mes',
 'Porto Atracação',
 'SGUF']

In [100]:
# Teste Final para colocar a informação do peso liquido na tabela carga_fato
#carga_fato_tmp1.IDCarga == tb_carga_liqui.IDCarga, "inner" == ['IDCarga']
carga_fato = carga_fato_tmp1.join(tb_carga_liqui, ['IDCarga'] )

In [104]:
carga_fato.columns

['IDCarga',
 'IDAtracacao',
 'Origem',
 'Destino',
 'CDMercadoria',
 'Tipo Operação da Carga',
 'Carga Geral Acondicionamento',
 'ConteinerEstado',
 'Tipo Navegação',
 'FlagAutorizacao',
 'FlagCabotagem',
 'FlagCabotagemMovimentacao',
 'FlagConteinerTamanho',
 'FlagLongoCurso',
 'FlagMCOperacaoCarga',
 'FlagOffshore',
 'FlagTransporteViaInterioir',
 'Percurso Transporte em vias Interiores',
 'Percurso Transporte Interiores',
 'STNaturezaCarga',
 'STSH2',
 'STSH4',
 'Natureza da Carga',
 'Sentido',
 'TEU',
 'QTCarga',
 'VLPesoCargaBruta',
 'Ano',
 'Mes',
 'Porto Atracação',
 'SGUF',
 'Peso Liquido']

In [107]:
#Criação da Tabela 'carga_fato.csv' no diretório atual da execução do Jupyter / se desejar outro Path basta colocar na string
atracacao_fato.toPandas().to_csv('carga_fato.csv')

In [102]:
######## Tabela carga_fato####### Resumo das Operações
#Criação da tabeça carga_fato
# Campo comun entre as duas tabelas (tb_atr.IDAtracacao == tb_temp_atr.IDAtracacao)
# Inner - Combina os valores comuns dos mesmos campos nas duas tabelas (Intersecção)

# criar uma sessão usanda para extração da tabela carga_fato
spark = SparkSession.builder.appName("ANTAQ.carga_fato").getOrCreate()

#carga_fato = tb_atr.join(tb_temp_atr, tb_atr.IDAtracacao == tb_temp_atr.IDAtracacao, "inner" )
#Filtro da tabela de atracação e crga conteinerizada
tb_atr_filtro = tb_atr.select('IDAtracacao','Ano','Mes','Porto Atracação', 'SGUF')
tb_carga_contz_filtro = tb_carga_contz.select('IDCarga','VLPesoCargaConteinerizada')
tb_carga_filtro = tb_carga.select('IDCarga','VLPesoCargaBruta')

#Procedimento para pegar os valores de modo que : Carga não conteinerizada = Peso bruto e Carga 
#conteinerizada = Peso sem contêiner - 
tb_carga_liqui1 = tb_carga_filtro.join(tb_carga_contz_filtro, tb_carga_contz_filtro.IDCarga == tb_carga_filtro.IDCarga, "inner" )
tb_carga_liqui2 = tb_carga_liqui1.withColumn('Peso Liquido',tb_carga_liqui1['VLPesoCargaBruta'] + tb_carga_liqui1['VLPesoCargaBruta']  )
tb_carga_liqui = tb_carga_liqui2.select('IDCarga','Peso Liquido')

#Procedimentos de junções finais
# tb_carga.IDAtracacao == tb_atr_filtro.IDAtracacao, "inner" == ['IDAtracacao']
carga_fato_tmp1 = tb_carga.join(tb_atr_filtro, ['IDAtracacao'] )

#carga_fato_tmp1.IDCarga == tb_carga_liqui.IDCarga, "inner" == ['IDCarga']
carga_fato = carga_fato_tmp1.join(tb_carga_liqui, ['IDCarga'] )


AnalysisException: Reference 'IDCarga' is ambiguous, could be: IDCarga, IDCarga.

In [134]:
tb_atr.select('Ano','Mes','Porto Atracação', 'SGUF')

DataFrame[Ano: string, Mes: string, Porto Atracação: string, SGUF: string]

In [104]:
# Automatizar a extração
# ler o arquivo .csv/.txt com o cabeçalho e esquema para os anos 2018, 2019 e 2020

import os.path
# Carga da tabela de AcordosBilaterais
def Carga_ANTAQ(tb_ab):
    ano = ["2018","2019","2020"]
    tb_ab = pyspark.sql.dataframe.DataFrame()
    for i in ano:
        # verifica se o arquivo existe no dietório
        if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+i+"/"+i+"AcordosBilaterais.txt"):
            tb_ab = tb_ab.union(spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+i+"/"+i+"AcordosBilaterais.txt", header=True, sep=';'))
            #rdd = tb_ab.union(rdd)
            #tb_ab = tb_ab.union(rdd)
return tb_ab

SyntaxError: 'return' outside function (<ipython-input-104-6cff80ae00fa>, line 15)

In [45]:

tb_acbi_2018 = sc.(self, 'D:\Aulas 2021.1\SFIEC Seleção\Dados\2018\2018AcordosBilaterais.txt')  

NameError: name 'self' is not defined

In [28]:
tb_acbi_2018.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/D:/Aulas 2021.1/SFIEC Seleção/Dados88AcordosBilaterais.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:29

In [25]:
## Automatizar a extração para 2020
#antaq2020 = zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/2020.zip');
#w = zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/2020.zip') as antaq2020;
    ##print(*antaq2020.namelist(), sep="\n")
   
#with zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/2019.zip') as antaq2019:
#with zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/2018.zip') as antaq2018:
#with zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/MetadadosFrota.zip') as meta_frota:
#with zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/MetadadosAfretamento.zip') as meta_afreta:
#with zipfile.ZipFile('http://web.antaq.gov.br/Sistemas/ArquivosAnuario/Arquivos/MetadadosMovimentacao.zip') as meta_movi:

In [93]:
ano = ["2018","2019","2020"]
for i in ano:
    print(i)

2018
2019
2020


In [109]:
#Testes
ano = ["2018","2019","2020"]
tb_ab = pyspark.sql.dataframe.DataFrame()
for i in ano:
  # verifica se o arquivo existe no dietório
  ##if os.path.exists("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+i+"/"+i+"AcordosBilaterais.txt"):
    tb_ab = tb_ab.union(spark.read.csv("D:/Aulas 2021.1/SFIEC_Selecao/Dados/"+i+"/"+i+"AcordosBilaterais.txt", header=True, sep=';'))



TypeError: __init__() missing 2 required positional arguments: 'jdf' and 'sql_ctx'